In [1]:
from itertools import combinations 
from random import shuffle 
import numpy as np 
import sys 

In [2]:
# general stuff that will work for any card game 

class Card:
    
    def __repr__( self ):
        return '{}{}'.format( self.display_rank, self.display_suit )
    
    def __init__( self, suit, rank ):
        display_suits = [ 's', 'h', 'c', 'd' ]
        display_ranks = [ 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K' ]
        vals = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10 ]
        map_suits = dict(zip(range(4),display_suits))
        map_ranks = dict(zip(range(13),display_ranks))
        map_vals = dict(zip(range(13),vals))

        self.suit = suit
        self.rank = rank 
        self.index = (13*self.suit)+self.rank
        self.display_rank = map_ranks[ self.rank ]
        self.display_suit = map_suits[ self.suit ]
        self.value = map_vals[ self.rank ]
        
class Deck:
    def __init__( self ):
        self.cards = []
        for s in range( 4 ):
            for r in range( 13 ):
                self.cards.append(Card(s, r))
                
    def shuffle( self ):
        shuffle( self.cards )
        
    def draw( self, n ):
        result = []
        for i in range( n or 1 ):
            result.append( self.cards.pop() ) 
        return result 

In [3]:
deck = Deck()
deck.shuffle()
deck.draw(5)

[Kc, 4h, Ks, 7h, 9s]

In [4]:
def score( hand ):
    # must be a 5 card hand to be valid 
    if len( hand ) == 5:
        
        # first, look for cards that add up to 15 
        
        points = 0
        
        # fifteens  
        for vector_length in [ 2, 3, 4, 5 ]:
            for vector in combinations( hand, vector_length ):
                if sum( [ card.value for card in vector ] ) == 15:
                    points += 2 
                    
        # pairs (not necessary to account for more than pairs)
        for i, j in combinations( hand, 2 ):
            if i.rank == j.rank:
                points += 2 
                
        # runs
        for vector_len in [ 5, 4, 3 ]:
            for vec in combinations( hand, vector_len ):
                vals = [ card.value for card in vec ]
                run = [ n + min( vals ) for n in range( vector_len ) ]
                if sorted( vals ) == run:
                    points += vector_len
                    break
                break 
                      
    return points 

In [5]:
deck = Deck()
deck.shuffle()
for n in range( 5 ):
    hnd = deck.draw(5)
    print(score(hnd), hnd)

2 [Ks, 10h, 10c, 6s, 4h]
0 [9s, Kc, 10d, Ac, 2d]
0 [Jh, 7s, 10s, As, 2h]
4 [9h, 2c, 3c, Jd, 6d]
4 [5h, Qh, 8s, Ah, 8d]


In [6]:
# # check to see if it's possible to make a score of 19 
# for n in range( 10000 ):
#     deck = Deck()
#     deck.shuffle()
#     hand = deck.draw(5)
#     if score( hand ) == 19:
#         print( 'Uh oh', hand ) 

In [7]:
# all possible hands
all_possible_hands = combinations( deck.cards, 5 ) 

In [8]:
def play_a_hand():
    
    deck = Deck()
    deck.shuffle()
    
    p0_hand = deck.draw(6)
    p1_hand = deck.draw(6)
    turn_card = deck.draw(1)
    crib = []
    
    # player discard 
    crib.append( p0_hand.pop() ) 
    crib.append( p0_hand.pop() ) 
        
    # computer discard 
    crib.append( p1_hand.pop() )
    crib.append( p1_hand.pop() )
        
    #turn card and count hands 
    turn_card = deck.draw(1)
    p0_score = score( p0_hand + turn_card ) 
    p1_score = score( p1_hand + turn_card ) 
    
    # counting
    plays = [] # list of tuples, format: (player, card) 
    for i in range( 4 ):
        plays.append( p0_hand[i] )
        plays.append( p1_hand[i] )
    
    #play_vector = list(p0_hand) + list(p1_hand) + crib + list(turn_card) + plays
    #return ( p0_score, p1_score ), [ card.index for card in play_vector ]
    
    print('Player 0:', p0_hand)
    print('Player 1:', p1_hand)
    print('Crib (P2):', crib)
    print('Counting:', plays) 
    print('Final score:', ( p0_score, p1_score )) 
    
    return p0_score, p1_score

play_a_hand()

Player 0: [Ah, 10h, 5d, Ad]
Player 1: [4d, 10d, Qc, 6h]
Crib (P2): [2c, As, 7s, 7h]
Counting: [Ah, 4d, 10h, 10d, 5d, Qc, Ad, 6h]
Final score: (4, 2)


(4, 2)

In [9]:
def play_a_game():
    '''
    Plays a game of cribbage. Returns a winner, 0 or 1
    '''
    sc = np.array([0,0]) 
    while True:
        print( '>>> Game score:', sc )
        outcome = play_a_hand()
        sc += np.array(outcome)
        for i, p in enumerate( sc ):
            if p > 120:
                return i 
        
play_a_game()

>>> Game score: [0 0]
Player 0: [Qh, Qs, 6c, 2d]
Player 1: [3d, 10d, 3s, Kd]
Crib (P2): [5d, 8h, Jc, 7s]
Counting: [Qh, 3d, Qs, 10d, 6c, 3s, 2d, Kd]
Final score: (2, 4)
>>> Game score: [2 4]
Player 0: [4s, 8c, 8s, Ac]
Player 1: [Qc, 2d, 5s, 2s]
Crib (P2): [Qh, 4d, 8h, Qs]
Counting: [4s, Qc, 8c, 2d, 8s, 5s, Ac, 2s]
Final score: (6, 6)
>>> Game score: [ 8 10]
Player 0: [6d, 8c, Ac, 9c]
Player 1: [6s, 3h, 3c, 3d]
Crib (P2): [Kd, 4d, 10c, 2d]
Counting: [6d, 6s, 8c, 3h, Ac, 3c, 9c, 3d]
Final score: (4, 8)
>>> Game score: [12 18]
Player 0: [4d, 4s, 9c, 3h]
Player 1: [5d, 2h, As, 5s]
Crib (P2): [Qh, Jh, 10s, 10c]
Counting: [4d, 5d, 4s, 2h, 9c, As, 3h, 5s]
Final score: (4, 6)
>>> Game score: [16 24]
Player 0: [2h, Ac, 9d, 7s]
Player 1: [Ks, Jd, Qc, Kh]
Crib (P2): [3c, Jc, Kc, 10s]
Counting: [2h, Ks, Ac, Jd, 9d, Qc, 7s, Kh]
Final score: (0, 4)
>>> Game score: [16 28]
Player 0: [Ac, 3d, 5d, 4c]
Player 1: [3c, Jh, 3h, 8h]
Crib (P2): [9h, Jd, Qd, 7d]
Counting: [Ac, 3c, 3d, Jh, 5d, 3h, 4c, 8h]
Fina

1

In [10]:
# # play 10000 games 

# features = []
# target = []
# n_games = 10
# for n in range( n_games ): 
#     outcome, play_vector = play_a_game()
#     target.append( outcome ) 
#     features.append( play_vector ) 
#     sys.stdout.write("\rPlaying game {} of {} games".format( n+1, n_games ) )
#     sys.stdout.flush()
# sys.stdout.write("\rPlayed {0}/{0} games. Done.  ".format( n_games ) )
# sys.stdout.flush()

In [11]:
# %matplotlib inline 
# import matplotlib.pyplot as plt
# from sklearn import preprocessing, metrics, svm, linear_model, model_selection

In [12]:
# features = np.array(features)
# # features.shape

In [13]:
# one_hot = preprocessing.OneHotEncoder()
# transformed = one_hot.fit_transform( features ) 
# transformed.shape

In [14]:
# for i in range( 10 ):
#     X_train, X_test, y_train, y_test = model_selection.train_test_split(transformed, target, test_size=0.33)
#     clf = svm.SVC(kernel='linear',probability=True)
#     clf.fit( X_train, y_train ) 
#     pred = clf.predict( X_test ) 
#     prob = clf.predict_proba( X_test )
#     fpr, tpr, __ = metrics.roc_curve( y_test, prob[:,0] ) 
#     plt.plot( fpr, tpr ) 

1. First, see if there's a mvove th computer can make that will win the game (i.e. make the score 31). If there is, take that move. Otherwise, go to setp 2. 

2. See if there's a move the player could make that would cause the computer to lose the game (i.e. play a card such that computer cannot play). If there is, block that move. 

3. Play a predefined good/safe card. 

4. Play a different predefined safe card. 

5. Play the only legal card. 

6. Say "go" . 

In [16]:
def game():
    
    # game functions 
    
    # universal play function 
    def play( card ):
        nonlocal count, turn, hands
        plays.append( card ) 
        count += card.value
        hands[turn] = [ n for n in hands[turn] if n != card ]         
        print( card, 'was played by player', turn ) 
        return count, hands 

    # human choosing function 
    def ask_for_input():
        nonlocal hands
        d = dict(enumerate(hands[1],1))
        inp = input( 'Choose one of {}'.format( d ) )
        card = d[ int(inp) ]
        return card 

    # AI choosing function 
    def ai_figure_out_which_card_to_play():
        nonlocal hands, count, score, turn     
        choices = hands[0] # list of 4 choices 
        return choices[0]
        
    #
    def say_go():
        nonlocal score 
        pass 
    
    def validate_play( card, count ):
        if card.value + count < 32:
            return True 
        else:
            return False 
    
    # game logic         
    deck = Deck()
    deck.shuffle()        
    hands = [deck.draw(4), deck.draw(4)]
    turn = np.random.randint(2) # either 0 or 1 
    count = 0
    plays = []
    score = np.array([0,0]) # get a player's score like score[player_number] 

    while all(hands):
        game_on = True
        while game_on: # we are counting toward 31 
            print( 'Count is {}'.format( count ) ) 
            print( 'Score is {}'.format(tuple(score)) ) 

            # first, does whomever's turn it is even have cards? 
            if not hands[turn]:
                break # out of while game_on 
            # first, is whoever's turn it is even have a legal move
            # or do they have to say "go"? 
            if all([count+card.value>31 for card in hands[turn]]):
                # they have to say "go" 
                # other person gets 1 point 
                score[turn^1] += 1 
                print( "Go --- player", turn ) 
                break # out of "while game_on" 

            # player's turn 
            if turn == 1:
                done = False
                while not done: 
                    card = ask_for_input()
                    done = validate_play(card, count)
                play(card) 

            # ai turn 
            else:
                done = False
                while not done: 
                    card = ai_figure_out_which_card_to_play()
                    done = validate_play(card, count)
                play(card) 

            # score and set up for next iteration 
            # current player is 
            if count == 31:
                # we are done, son 
                score[ turn ] += 2 # 31 for 2 
                break 
            if count == 15:
                score[ turn ] += 2 # 15 for 2 

            # swap turn     
            turn ^= 1 # swap turn 

        # anything you want to happen after we stop couting to 31 
        print( "Plays to 31:", plays ) 
        count = 0 
#         if all(hands):
#             game_on = True 
#         else: 
#             game_on = False 
    # anything you want to happen after we stop the game!  

game()

Count is 0
Score is (0, 0)
Jh was played by player 0
Count is 10
Score is (0, 0)
Choose one of {1: Qs, 2: 8c, 3: 2h, 4: 5c}1
Qs was played by player 1
Count is 20
Score is (0, 0)
2s was played by player 0
Count is 22
Score is (0, 0)
Choose one of {1: 8c, 2: 2h, 3: 5c}1
8c was played by player 1
Count is 30
Score is (0, 0)
Go --- player 0
Plays to 31: [Jh, Qs, 2s, 8c]
Count is 0
Score is (0, 1)
Kh was played by player 0
Count is 10
Score is (0, 1)
Choose one of {1: 2h, 2: 5c}2
5c was played by player 1
Count is 15
Score is (0, 3)
4c was played by player 0
Count is 19
Score is (0, 3)
Choose one of {1: 2h}1
2h was played by player 1
Count is 21
Score is (0, 3)
Plays to 31: [Jh, Qs, 2s, 8c, Kh, 5c, 4c, 2h]
